## Loading Parquet Files as External Tables in Netezza  

Its possible your system is missing some python libraries. If when you try to import the various libraries below a "Module Not Found" error is returned. Use the pip install commands to load the proper libraries on your system. 

In [ ]:
###
###
!pip install nzpy 

# Import Libraries 
import nzpy
import datetime

In [ ]:
#Netezza Cloud Connection Information 

nz_host             = ""
nz_port             = 5480
nz_database         = ""
nz_user             = ""
nz_password         = ""


In [ ]:
# Create nzpy connection to cloud instance 

nzcon = nzpy.connect(user=nz_user, password=nz_password, host=nz_host, database=nz_database, port=nz_port)


In [ ]:
# Create the external tables for the historical data stored in Parquet files

with nzcon.cursor() as cursor:
    years = ["2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
    cursor.execute("set ENABLE_EXTERNAL_DATASOURCE = 1")
    for year in years:
        ddl="create EXTERNAL table ext_airline_delay_cause_" + year + " on cosdb.admin.airline_cos_datasource using ( DATAOBJECT ('/airline/airline_delay_cause_" + year + ".parquet') FORMAT 'PARQUET');"
        try:
            cursor.execute(ddl)
            print("Data Lakehouse table EXT_AIRLINE_DELAY_CAUSE_" + year + " created successfully")
        except Exception as e:
            print(str(e))

In [ ]:
# Create a single external table for all historical flight delay data

ddl="""create EXTERNAL table airline_delay_cause_history on cosdb.admin.airline_cos_datasource
    using ( DATAOBJECT ('/airline/airline_delay_cause_history_2003_2018.parquet') FORMAT 'PARQUET')"""

with nzcon.cursor() as cursor:
    cursor.execute("set ENABLE_EXTERNAL_DATASOURCE = 1")
    try:
        cursor.execute(ddl)
        print("Data Lakehouse external table AIRLINE_DELAY_CAUSE_HISTORY created successfully")
    except Exception as e:
            print(str(e))

In [ ]:
# Create Netezza Native table for years 2018 from parquet external table

ddl="""create table demo.airline_delay_cause_2018_local
as select * from demo.ext_airline_delay_cause_2018
distribute on random
organize on (year, month, carrier, airport)
;
"""

with nzcon.cursor() as cursor:
    try:
        cursor.execute(ddl)
        print("Table AIRLINE_DELAY_CAUSE_2018_LOCAL created successfully.")
    except Exception as e:
        print(str(e))
